###### Reduced Events Table

__Description__: Reduced excess rainfall is calculated for each event using a user-specified stormwater removal rate and capacity.

__Input__: A JSON file containing the incremental excess rainfall for a suite of events which may have different durations and boundary condition names, and the stormwater removal rate and capacity. 

__Output__: 

- The unreduced incremental excess rainfall for each event as a JSON.

- The incremental reduced excess rainfall and lateral inflow hydrographs for each event as a JSON.

- A metadata file containing the stormwater removal rate, the stormwater capacity, and the seed of the random number generator.

---

## A. Load Libraries, Parameters, and Data:

### Libraries:

In [1]:
import sys
sys.path.append('core')
import hydromet_reduced
from hydromet import*

### Parameters: 

#### Papermill (site specific):

In [2]:
## Filenames and paths:
Project_Area = 'Passaic'   # Project area name
Pluvial_Model = 'P01'         # Pluvial model name
BCN = 'D01'                   # Domain/boundary condition name

pluvial_params = '{0}_{1}_Pluvial_Parameters.xlsx'.format(Project_Area, Pluvial_Model) 
rate_column = 'SW Rate (in/30min)'
capacity_column = 'SW Capacity (in)'

#root_dir = pl.Path(os.getcwd())
root_dir = pl.Path(r'C:\Users\sputnam\Desktop\PFRA_Production\Pluvial\Test2')
outputs_dir = root_dir/'Outputs'
inputs_dir = root_dir/'Inputs'
pluvial_params_dir = inputs_dir/pluvial_params


## Options:
display_plots = True
display_print = True
run_dur_dic = {'6': 1, '12': 1, '24': 2, '96': 5}  # Run durations in days

##### Convert all paths to objects:

In [3]:
outputs_dir = pl.Path(outputs_dir)
pluvial_params_dir = pl.Path(pluvial_params_dir)

##### Set the base filestem for reading/writing files:

In [4]:
filestem = '{0}_{1}_{2}'.format(Project_Area, Pluvial_Model, BCN)

### Data:
#### Stormwater removal rate and capacity:

In [5]:
rate, maxcap = get_stormwater_rate_cap(pluvial_params_dir, BCN, rate_column, capacity_column, display_print)

,Pluvial Domain,Shapefile,Curve Number,SW Infrastructure (YES or NO),SW Rate (in/30min),SW Capacity (in),Comments,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,D01,P01_2D_Domain.shp,82.0,YES,0.1083,8.47,SW Rate from NJDOT 2015 Roadway Design Manual,NaN,NaN,NaN,NaN,NaN,NaN


None
SW Rate: 0.1083 in/30min
SW Capacity: 8.47 in/unit area


#### Lateral inflow domains:

In [ ]:
lid = get_lateral_inflow_domains(pluvial_params_dir, BCN, display_print)

#### Excess rainfall data:

In [ ]:
with open(outputs_dir/'{}.json'.format(filestem)) as f:
    EventsTable = json.load(f)

##### Extract the durations:

In [ ]:
durations = list(EventsTable.keys())
print('Durations:', durations)

---

## B. Calculate Reduced Excess Rainfall:

In [ ]:
ReducedTable, StormwaterTable, SW_variables = hydromet_reduced.main(EventsTable, durations, BCN, rate = rate, maxcap = maxcap, 
                                                                    display_print = display_print, display_plots = display_plots)

---

## C. Calculate the Lateral Inflow Hydrographs:

In [ ]:
if lid.shape[0]>0:
    ReducedTable = calc_lateral_inflow_hydro(lid, ReducedTable, StormwaterTable, durations, BCN, display_print, display_plots, plot_lid_num = 0)

---

## D. Save the Results:

### Unreduced excess rainfall:

In [ ]:
with open(outputs_dir/'{0}_Unreduced.json'.format(filestem), 'w') as f:
    json.dump(EventsTable, f)

### Reduced excess rainfall:

In [ ]:
with open(outputs_dir/'{}.json'.format(filestem), 'w') as f:
    json.dump(ReducedTable, f)

### Metadata:

In [ ]:
with open(outputs_dir/'{0}_SWMetadata.json'.format(filestem), 'w') as f:
    json.dump(SW_variables, f)

---

## END